In [252]:
molecule_mask = torch.tensor(ent_type[:,0][G_tg.edge_index[0]] == 1)
ent_type = torch.tensor(np.load(f'{data_path}/ent_type_onehot.npy'))

nodes, _, _, edge_mask = k_hop_subgraph(0, 1, G_tg.edge_index)
double_mask = molecule_mask * edge_mask
mask_idx = torch.where(double_mask)[0]
edge_subgraph = G_tg.edge_subgraph(mask_idx)
subgraph = edge_subgraph.subgraph(nodes)
masked_node_ids = edge_subgraph.edge_index[0] # (num_masked_nodes,)
motif_labels = motifs[masked_node_ids] # (num_masked_nodes, motif_len)
node_labels = ent_type[masked_node_ids] # (num_masked_nodes, num_ent_type)
rel_labels = subgraph.relation

subgraph, masked_node_ids.shape, motif_labels.shape, node_labels.shape, rel_labels.shape

(Data(edge_index=[2, 1951], relation=[1951], num_nodes=385),
 torch.Size([1951]),
 torch.Size([1951, 85]),
 torch.Size([1951, 16]),
 torch.Size([1951]))

In [251]:
rel_labels

tensor([ 0,  0,  0,  ..., 31, 31, 31])

In [238]:
print(G_tg.edge_index[:, mask_idx][0])

tensor([    0,     0,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,    29,    29,    29,    29,   661,   661,   661,   661,   661,
          661,   661,   661,   661,   661,  2277,  2343,  2343,  2343, 18686,
        18686, 18686, 18686, 18686, 18686, 18686, 28000, 28000, 28000, 28749,
        29943, 29943, 29943, 29943, 29943, 29943, 33784, 33784, 33784, 33784,
        33784, 39941, 39941, 39941, 39941, 39941, 39941, 42393, 42393, 42393,
        42393, 42393, 42393, 42393, 42393, 42393, 42624, 42624, 42624, 42624,
        42624, 42624, 43274, 43462, 48532, 48532, 48532, 48532, 48532, 48532,
        58132, 58132, 58132, 58132, 58132, 58132, 58132, 60341, 65188])


In [239]:
subgraph.edge_index

tensor([[ 0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,
          5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  8,  9,  9,  9,  9,
          9,  9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12,
         12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 14, 15, 16, 16, 16, 16, 16, 16,
         17, 17, 17, 17, 17, 17, 17, 18, 19],
        [ 1,  7,  0,  2,  4,  5,  3, 18,  8,  6, 14, 11, 16,  7, 15, 13,  9, 10,
         17, 19, 12,  1,  2,  5,  3,  1,  2, 16, 11, 12, 10,  6, 13,  9, 17,  1,
          1,  2,  7,  1,  3, 13,  9, 17, 12, 11,  0,  1,  5,  1,  1,  3,  6, 13,
         17, 12,  1,  3, 11, 16, 12,  1,  3,  6, 10, 16, 12,  1,  3,  6,  9, 10,
         11, 16, 13, 17,  1,  3,  6,  9, 12, 17,  1,  1,  1,  3, 10, 11, 12, 17,
          1,  3,  6,  9, 12, 13, 16,  1,  1]])

In [107]:
s1.number_of_nodes(), s1.number_of_edges()

(3433, 3432)

In [94]:
for c_id in tqdm(center_molecule_ids):
    subgraph = s1 = get_subgraph(G, c_id)
    

16979

In [1]:
from pretrain import *

# your credentials

params = {
    "lr": 1e-4,
    "hidden_dim": 200,
    "epochs": 100,
    "lambda": "08_15_10"
}
logger = get_logger(lr=params['lr'], hidden_dim=params['hidden_dim'], epochs=params['epochs'], lambda_=params['lambda'])


# Data path
data_path = '../data_process/pretrain_data'
print('Getting everything prepared...')
ent_type, motifs, G_tg, center_molecule_ids, molecule_mask = get_everything(data_path)

# Load KGE embeddings
# return:
# entity_embedding: (num_ent, emb_dim)
# relation_embedding: (num_rel, emb_dim)
emb_path = '/data/pj20/molkg_kge/transe'
print('Loading KGE embeddings...')
entity_embedding, relation_embedding = load_kge_embeddings(emb_path)

# Initialize model
print('Initializing model...')
model = KGNN(
    node_emb=entity_embedding,
    rel_emb=relation_embedding,
    num_nodes=ent_type.shape[0],
    num_rels=39,
    embedding_dim=512,
    hidden_dim=200,
    num_motifs=motifs.shape[1],
    lambda_edge=0.8,
    lambda_motif=1.5,
    lambda_mol_class=1
)

# Train
device = torch.device('cuda:1')
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 100

/home/pj20/miniconda3/envs/kgc/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Getting everything prepared...
Loading entity type labels...
Loading center molecule motifs...
Loading entire knowledge graph...


/home/pj20/miniconda3/envs/kgc/lib/python3.8/site-packages/torch_geometric/utils/convert.py:250: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  data[key] = torch.tensor(value)
/home/pj20/gode/k_level_pretrain/pretrain.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  molecule_mask = torch.tensor(ent_type[:,0][G_tg.edge_index[0]] == 1) # (num_edges,)


Loading molecule mask...
Loading KGE embeddings...
Loading KGE embeddings...
Initializing model...


In [3]:
# Get dataloader
train_loader, val_loader = get_dataloader(G_tg, center_molecule_ids, molecule_mask, motifs, ent_type, batch_size=4)
train_loop(model, train_loader, val_loader, optimizer, device, epochs)

loss: 0:   0%|          | 0/7605 [00:00<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [4]:
for batch in train_loader:
    print(batch)
    break

DataBatch(edge_index=[2, 122], relation=[122], num_nodes=22, masked_node_ids=[122], rel_ids=[122], center_molecule_id=[1], motif_labels=[122, 85], node_labels=[122, 16], batch=[22], ptr=[2])


In [4]:
torch.zeros((1, 39)).shape

torch.Size([1, 39])

In [6]:
torch.zeros(39).reshape(1, -1).shape

torch.Size([1, 39])

In [13]:
torch.where(torch.tensor([0, 1, 4, 5]) == 4)[0][0].int()

tensor(2, dtype=torch.int32)

In [85]:
import torch
from pretrain import *

def get_everything(data_path):
    # Training Labels
    ## Load entity type labels
    print('Loading entity type labels...')
    ent_type = torch.tensor(np.load(f'{data_path}/ent_type_onehot.npy')) # (num_ent, num_ent_type)

    ## Load center molecule motifs
    print('Loading center molecule motifs...')
    motifs = []
    with open(f'{data_path}/id2motifs.json', 'r') as f:
        id2motifs = json.load(f)
    motif_len = len(id2motifs['0'])
    for i in range(len(ent_type)):
        if str(i) in id2motifs.keys():
            motifs.append(np.array(id2motifs[str(i)]))
        else:
            motifs.append(np.array([0] * motif_len))

    motifs = torch.tensor(np.array(motifs), dtype=torch.long) # (num_ent, motif_len)

    ## Center molecule ids
    center_molecule_ids = torch.tensor([int(key) for key in id2motifs.keys()])

    # Entire Knowledge Graph (MolKG)
    print('Loading entire knowledge graph...')
    G = nx.read_gpickle(f'{data_path}/graph.gpickle')
    G_tg = from_networkx(G)

    # molecule_mask
    print('Loading molecule mask...')
    molecule_mask = torch.tensor(ent_type[:,0][G_tg.edge_index[0]] == 1) # (num_edges,)

    return ent_type, motifs, G_tg, center_molecule_ids, molecule_mask

In [86]:
ent_type, motifs, G_tg, center_molecule_ids, molecule_mask = get_everything("../data_process/pretrain_data")

Loading entity type labels...
Loading center molecule motifs...
Loading entire knowledge graph...
Loading molecule mask...


/tmp/ipykernel_107140/1126395188.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  molecule_mask = torch.tensor(ent_type[:,0][G_tg.edge_index[0]] == 1) # (num_edges,)


In [74]:
molecule_mask = torch.tensor(ent_type[:,0] == 1) # (num_edges,)

/tmp/ipykernel_107140/2937019473.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  molecule_mask = torch.tensor(ent_type[:,0] == 1) # (num_edges,)


In [75]:
torch.where(molecule_mask)[0].shape

torch.Size([88811])

In [22]:
from torch_geometric.utils import k_hop_subgraph

nodes, _, _, edge_mask = k_hop_subgraph(int(center_molecule_ids[0]), 1, G_tg.edge_index)

In [18]:
double_mask = molecule_mask * edge_mask

In [26]:
len(torch.where(edge_mask)[0])

1192001

In [131]:
data_path = '/data/pj20/molkg/pretrain_data'
with open(f'{data_path}/graph.pt', 'rb') as f:
    G_tg = torch.load(f)

In [134]:
G_tg.edge_index[1][1332817]

tensor(70403)

In [193]:
molecule_head_mask = ent_type[:,0][G_tg.edge_index[0]] == 1 # (num_edges,)
non_molecule_head_mask = ent_type[:,0][G_tg.edge_index[0]] == 0 # (num_edges,)
molecule_tail_mask = ent_type[:,0][G_tg.edge_index[1]] == 1 # (num_edges,)
non_molecule_tail_mask = ent_type[:,0][G_tg.edge_index[1]] == 0 # (num_edges,)

mol2mol_mask = molecule_head_mask * molecule_tail_mask
mol2nonmol_mask = molecule_head_mask * non_molecule_tail_mask
nonmol2mol_mask = non_molecule_head_mask * molecule_tail_mask

tensor([False, False, False,  ..., False, False, False])

In [104]:
len(torch.where(mol2nonmol_mask)[0])

916534

In [105]:
ent_type.shape

torch.Size([184819, 16])

In [39]:
molecule_mask[center_molecule_ids[10000]]

tensor(True)

In [135]:
center_molecule_ids[1]

tensor(43924)

In [138]:
from torch_geometric.utils import k_hop_subgraph

nodes, _, _, edge_mask = k_hop_subgraph(int(center_molecule_ids[1]), 0, G_tg.edge_index)

In [190]:
from torch_geometric.utils import to_undirected, k_hop_subgraph

def get_k_hop_nodes(node_index, num_hops, edge_index):
    # Convert the edge_index to undirected
    edge_index = to_undirected(edge_index)

    # Compute the k-hop subgraph
    node_idx_k_hop, _, _, _ = k_hop_subgraph(node_index, num_hops, edge_index, relabel_nodes=False, num_nodes=None, flow='source_to_target')

    return node_idx_k_hop

In [218]:
node_ids = get_k_hop_nodes(int(center_molecule_ids[8]), 1, G_tg.edge_index)

In [225]:
G_tg.subgraph(torch.tensor([4, 5, 6, 7, 8])).edge_index

tensor([[1, 3, 4, 1, 4, 4, 0, 1, 4],
        [2, 2, 2, 3, 3, 1, 1, 3, 2]])

In [221]:
sub_1

tensor([[ 2,  4,  5,  4,  4,  3,  4,  5,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,
          4,  0,  2,  5,  4,  4,  4,  4,  4,  4,  4],
        [ 1, 12, 10, 11, 10, 10, 12,  0,  2,  2,  0,  1,  5,  3, 13, 17, 16, 14,
         15,  7,  6,  7,  7,  6, 18,  9,  8, 19, 20]])

In [129]:
G_tg.subgraph(torch.tensor([43924])).edge_index

/home/pj20/miniconda3/envs/kgc/lib/python3.8/site-packages/torch_geometric/data/storage.py:303: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'relation', 'edge_index', 'rel_label'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


tensor([], size=(2, 0), dtype=torch.int64)

In [130]:
len(torch.where(edge_mask)[0])

0

In [109]:
double_mask_mol2mol = mol2mol_mask * edge_mask
double_mask_mol2nonmol = mol2nonmol_mask * edge_mask

In [111]:
torch.where(double_mask_mol2mol)[0]

tensor([ 537424,  537430,  537431,  ..., 2100066, 2118004, 2119986])

In [82]:
center_molecule_ids[0]

tensor(1251)

In [92]:
ent_type[85600]

tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=torch.float64)

In [45]:
edge_mask.shape, molecule_mask.shape, double_mask.shape

(torch.Size([4666855]), torch.Size([4666855]), torch.Size([4666855]))

In [20]:
mask_idx = torch.where(double_mask == True)[0]

In [27]:
nodes

tensor([    10,     11,     12,  ..., 112293, 125440, 177470])

In [28]:
G_tg.subgraph(nodes)

Data(edge_index=[2, 1192001], relation=[1192001], rel_label=[1192001, 39], num_nodes=1637)

In [226]:
f'{True}'

'True'